**BERT 모델 불러오기**

In [1]:
from transformers import BertModel, BertTokenizer
import torch

# 1. BERT 토크나이저와 모델을 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [2]:
def specialtoken(tokenizer):
    special_tokens = {
        "unk_token": tokenizer.unk_token,
        "sep_token": tokenizer.sep_token,
        "pad_token": tokenizer.pad_token,
        "cls_token": tokenizer.cls_token,
        "mask_token": tokenizer.mask_token
    }
    return special_tokens

In [4]:
# tokenizer의 내장함수를 이용해 특수 토큰을 불러오는 함수를 작성해주세요
specialtoken(tokenizer)

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

**1.BERT Tokenizer의 특수 토큰**

아래 코드에서 특수 토큰을 불러오는 함수를 작성하고 각 토큰이 무엇을 의미하는지 적어주세요

- [UNK] : 알 수 없는 단어 토큰
- [SEP] : 구분 토큰
- [PAD] : 패딩 토큰
- [CLS] : 클래스 토큰
- [MASK] : 마스킹 토큰

**tokenizer를 이용해 문장을 토큰화**

토큰화를 통해 문자를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 한다.

In [5]:
# 입력 문장
sentence = "Hello, this is a sentence for tokenization."

# 문장 토큰화 및
inputs = tokenizer(sentence, return_tensors="pt")

# 토큰 ID를 다시 토큰으로 변환
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())  # 텐서를 리스트로 변환

print("Token IDs:", inputs['input_ids'])
print("Tokens:", tokens)

Token IDs: tensor([[  101,  7592,  1010,  2023,  2003,  1037,  6251,  2005, 19204,  3989,
          1012,   102]])
Tokens: ['[CLS]', 'hello', ',', 'this', 'is', 'a', 'sentence', 'for', 'token', '##ization', '.', '[SEP]']


**2. 토큰화**
토큰화 이후에 [CLS] 토큰과 [SEP] 토큰이 추가된 이유를 작성하세요

- [CLS] 토큰이 추가된 이유 : sequence 의 시작을 말해줌
- [SEP] 토큰이 추가된 이유 : 2개의 sentence를 구분해줌

**감성 분류를 위한 사전 학습된 BERT 모델을 이용해 감성분류하기**

In [11]:
import torch
from transformers import BertForSequenceClassification
from torch.nn.functional import softmax

# 감성 분류를 위한 사전 학습된 모델
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [39]:
# sentence에 대해 BERT를 이용해 감성분류 하는 함수
def sentiment_analysis(sentence):

    inputs = tokenizer(sentence, return_tensors="pt")

    input_ids = inputs['input_ids']

    outputs = model(input_ids)

    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)

    probs_list = probs.flatten().tolist()

    # 클래스 레이블 정의
    class_label = ['매우 부정적', '부정적', '중립적', '긍정적', '매우 긍정적']

    # 결과 출력 (소수점 2자리로 출력)
    for i, prob in enumerate(probs_list):
        print(f"{class_label[i]}일 확률: {prob:.2f}")

**3. BERT를 이용하여 각각 class에 맞는 sentence 작성해보기**

1. 매우 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
2. 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
3. 중립적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
4. 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
5. 매우 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요

In [46]:
# 매우 부정적인 문장 작성
sentence = "난 정말 멍청하고 부정적이고 쓸모없고 화나서 죽고싶다"

# 감성분석 수행
sentiment_analysis(sentence)


매우 부정적일 확률: 0.65
부정적일 확률: 0.25
중립적일 확률: 0.06
긍정적일 확률: 0.02
매우 긍정적일 확률: 0.02


In [47]:
# 부정적인 문장 작성
sentence = "너무 불만족스러워"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.30
부정적일 확률: 0.40
중립적일 확률: 0.21
긍정적일 확률: 0.05
매우 긍정적일 확률: 0.03


In [52]:
# 중립적 문장 작성
sentence = "내 기분은 적당히 그냥 그래"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.09
부정적일 확률: 0.33
중립적일 확률: 0.48
긍정적일 확률: 0.08
매우 긍정적일 확률: 0.02


In [53]:
# 긍정적 문장 작성
sentence = "오늘 맛있는거 먹어서 좋아"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.02
부정적일 확률: 0.03
중립적일 확률: 0.16
긍정적일 확률: 0.37
매우 긍정적일 확률: 0.42


In [54]:
# 매우 긍정적인 문장 작성
sentence = "진짜 오늘은 내 인생에서 최고의 날이야"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.06
부정적일 확률: 0.04
중립적일 확률: 0.09
긍정적일 확률: 0.13
매우 긍정적일 확률: 0.67


**4. BERT의 감성 분류 기준에 대해 생각을 적어보기**


BERT가 어떤 요소에 따라 감성 분류 기준을 나누는지에 대해 적어보세요

내가 생각하는 분류 기준 : 감정 분류 기준은 단어, 구문, 표현에 의해 기준이 나누어지는 것 같다. 긍정적이거나 부정적인 단어를 추가하면 그 확률이 확연하게 높아지는 결과가 나타났음.